<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [7]:
import os
import re
import warnings
import nltk
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
import string
from tqdm.notebook import tqdm
tqdm.pandas()

# Избавляемся от вывода предупреждений
warnings.simplefilter("ignore")

# Загружаем датасет
df_product = pd.read_csv('C:/Датасеты и проекты/marketing_product.csv', sep=';')


def dealer(dealer_name, text_to_find):
    data = df_product.dropna(subset=['name_1c', dealer_name]).reset_index(drop=True)
    data = data
    
    stemmer = nltk.stem.porter.PorterStemmer()

    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

    def stem_tokens(tokens):
        return [stemmer.stem(item) for item in tokens]    

    def normalize(text):    
        text = text.replace('\t', '\n').split('\n')
        text = " ".join(text)
        return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

    vectorizer = TfidfVectorizer(tokenizer=normalize)

    def cosine_sim(text1, text2):
        tfidf = vectorizer.fit_transform([text1, text2])
        return ((tfidf * tfidf.T).A)[0,1]
    
    # Промежуточная функция.

    def similarity (text):
        return cosine_sim(text_to_find, text)
    
    # Столбец 'similarity' покажет схожесть выбранного названия у диллера по отношению к названиям name_1c.
    
    data['similarity'] = data['name_1c'].progress_apply(similarity)
    
    # 0.9 это степень схожести
    id_numbers = data.loc[data['similarity'] >= 0.9, 'id'].tolist()
    print(f'Номера id товаров Procept(name_1c): {id_numbers}')


# Здесь 'wb_name' - колонка с названиями у дилера, df_product['wb_name'][5] - название товара у дилера (строка),там не должно быть пропусков
dealer('wb_name', df_product['wb_name'][10])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Михан\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/455 [00:00<?, ?it/s]

Номера id товаров Procept(name_1c): [241]
